In [20]:
import pandas as pd
import numpy as np
import math

## Creation of the FPTA 

In order to be able to make our predictions, we first need to create our prefix tree, representing all the possible sequences read by the tree. 

We start by creating the *TrieNode* class. It represents a node of our tree. It takes as attribute:
- a string of characters 'text', which represents the label of each node.
- an 'is_word' boolean initialized to False. It becomes True if it has no children (i.e. if the word is finished).
- a list of children, to which we initialize for all nodes the first element representing the number of words ending at this node (completes the boolean presented above)
- an integer 'state', which represents the id of the node. It is unique for each node of the tree.
- an 'is_displayed' boolean initialized to False. It is used when displaying the tree, when there are loops so that it is displayed only once.
- a list of precedents, which grows as you go. All the precedents of each node are added to it. For the root node, we consider that its predecessor is itself.

For the *PrefixTree* class, which represents the tree as such, we have:
- a *TrieNode* which represents the root node.
- class functions that we will detail below

### Class functions of the *PrefixTree*.

#### Insert function:
It allows us to insert a word (taken as a parameter), starting from a certain node (current parameter). To do so, we enumerate the word (each character is taken one by one with its corresponding index), then, if the character is not present in the list of children, we will create a node and then increment the indices. If there already exists a child node with the same character, we increment its frequency by 1. When the word is finished, we pass the boolean to True, and we increment the word counter by 1. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/be/Trie_example.svg/1200px-Trie_example.svg.png" alt="Drawing" style="width: 200px;"/>

In [21]:
class TrieNode:
    _COUNTER_NODE = 1
    def __init__(self, text = "λ"):
        self.text = text
        self.is_word = False
        self.children = list()
        self.children.append(['#', 0, self]) #1st element:SYMB 2nd element:FREQUENCY 3rd element:CHILDREN 
        self.state = TrieNode._COUNTER_NODE
        TrieNode._COUNTER_NODE += 1
        self.previous = []

    def get_frequency(self): # Not counting the number of times a word ends 
        freq = 0
        for child in self.children[1:]:
            freq += child[1]
        return freq
    
    def get_frequency2(self):
        freq = 0
        for child in self.children:
            freq += child[1]
        return freq

    def __str__(self):
        return 'symb:{} state:{} -> children:{}'.format(self.text, self.state, self.children)

class PrefixTree:
    def __init__(self):
        self.root = TrieNode()
        self.matrice_transition = pd.DataFrame()
        self.matrice_emission = pd.DataFrame()

    def insert(self, word, current = None):
        '''
        Creates a given word in the trie
        '''
        if not current: 
            current = self.root
            pre = self.root
            
        for i, char in enumerate(word):
            
            if char not in [x[0] for x in current.children]:
                prefix = word[0:i+1]
                current.children.append([char, 1, TrieNode(prefix)])
                
                if pre not in current.previous:
                    current.previous.append(pre)
                    
                pre = current
                current = current.children[-1][2]

            else:
                for child in current.children:
                    if child[0] == char:
                        child[1] += 1
                        
                        if pre not in current.previous:
                            current.previous.append(pre)
                            
                        pre = current
                        current = child[2]

        current.is_word = True   
        current.children[0][1] += 1
        
    def starts_with(self, prefix, current = None):
        '''
        Returns a list of all words beginning with the given prefix, or
        an empty list if no words begin with that prefix.
        '''
        words = list()
        if not current:
            current = self.root
        for char in prefix:
            if char not in [x[0] for x in current.children]:
                return list()
            for child in current.children:
                  if child[0] == char:
                    current = child[2]
        self.__child_words_for(current, words)
        return words
    
    def __child_words_for(self, node, words):
        '''
        Private helper function. Cycles through all children
        of node recursively, adding them to words if they
        constitute whole words (as opposed to merely prefixes).
        '''
        if node.is_word:
            words.append(node.text)
        for child in node.children[1:]:
            self.__child_words_for(child[2], words)
    
    def size(self, current = None, visited = None):
        '''
        Returns the size of this prefix tree, defined
        as the total number of nodes in the tree.
        '''
        # By default, get the size of the whole trie, starting at the root
        if not current:
            current = self.root
        
        if not visited:
            visited = []
            visited.append(current)
            
        count = 1
        
        for child in current.children[1:]:
            if(child[2] not in visited):
                visited.append(child[2])
                count += self.size(child[2], visited)
                
        return count

    def leaf(self, current = None, visited = None):
        if not current: 
            current = self.root
            
        if not visited:
            visited = []
            
        count = 0
        tmp = 0
        
        self.__child_words_for(current, count)        
        return count
    
    def __leaves_for(self, current, count, visited):
        if current.is_word == True:
            count += 1
        for child in current.children[1:]:
            if(child[2] not in visited):
                visited.append(child[2])
                self.__leaves_for(child[2], count, visited)
    
    def transform2proba(self, current = None):
        if not current:
            current = self.root
            
        freq = current.get_frequency2()
        for child in current.children:
            if(isinstance(child[1], int) == False):
                return 0
            else: 
                child[1] = child[1]/freq
                if child[2].state != current.state:
                    current = child[2]    
                    self.transform2proba(current)
                    
    def display(self):
        '''
        Prints the contents of this prefix tree.
        '''
        print('====================================================================================')
        self.__displayHelper(self.root)
        print('====================================================================================\n')
    
    def __displayHelper(self, current, visited = None):
        '''
        Private helper for printing the contents of this prefix tree.
        '''
        if not visited: 
            visited = []
            visited.append(current)
        
        print(current, "\n")
        
        for child in current.children[1:]:
            if child[2] not in visited:
                visited.append(current)
                self.__displayHelper(child[2], visited)

In [22]:
file = open("../1- Data Analysis/sequences.txt", "r")

seq = []
for line in file:
    stripped_line = line.strip()
    seq.append(stripped_line)

file.close()

In [23]:
trie = PrefixTree()
for seqq in seq:
    trie.insert(seqq)
    
trie.display()

symb:λ state:1 -> children:[['#', 0, <__main__.TrieNode object at 0x000001D9EDD0A588>], ['9', 14, <__main__.TrieNode object at 0x000001D9EDCE7648>], ['1', 24, <__main__.TrieNode object at 0x000001D9EDCE7948>], ['7', 11, <__main__.TrieNode object at 0x000001D9EBAEE948>], ['8', 89, <__main__.TrieNode object at 0x000001D9EDCE8388>], ['3', 20, <__main__.TrieNode object at 0x000001D9EDCE8E88>], ['0', 60, <__main__.TrieNode object at 0x000001D9EDCE86C8>], ['2', 33, <__main__.TrieNode object at 0x000001D9EDCE8D48>], ['4', 55, <__main__.TrieNode object at 0x000001D9EDCE8788>], ['6', 27, <__main__.TrieNode object at 0x000001D9EDCCAC48>], ['5', 38, <__main__.TrieNode object at 0x000001D9EDCCC348>]] 

symb:9 state:2 -> children:[['#', 0, <__main__.TrieNode object at 0x000001D9EDCE7648>], ['6', 3, <__main__.TrieNode object at 0x000001D9EDCE7FC8>], ['8', 5, <__main__.TrieNode object at 0x000001D9EDCCF3C8>], ['4', 2, <__main__.TrieNode object at 0x000001D9EDCD52C8>], ['5', 2, <__main__.TrieNode obje

## Creation of the DFA with the FPTA:

To transform a prefix tree into a frequency automaton (and thus probabilistic), a state-merging algorithm is used. Currently, the best one has been created by Colin de la Higuera. It is implemented from scratch below:

First of all, we have in parameters the sequences and an alpha parameter [0, 1]. 
We create the tree then we initialize 2 lists: red & blue: 
- Red stores all nodes that have already been defined as representative nodes and will be included in the final output model.
- Blue stores nodes that have not yet been tested.

The purpose of the algorithm is to keep only the red states. At the beginning,RED contains only the root node, whileBlue contains the immediate successor nodes of the initial node: the child nodes of the root. When executing the external loop of the algorithm, the first qb node in BLUE is chosen. If there is a qr node in RED compatible with qb, then qb and its successor nodes are merged with qr and the corresponding successor nodes of qr, respectively, as described in the Merge&Fold part. If qb is not compatible with any of the states of RED, it will be promoted, i.e. it will switch to RED, and its respective children will switch to BLUE.

#### Merge&Fold operation:
Given the DFA, the merge operation takes 2 states q and q′, where q is a RED state and q′ is a BLUE state compatible with the Hoeffding test. Merge operation consists of 2 steps. First, if q′est is a final state, then q becomes one as well (if it was not already the case) and the number of sequences ending in q′ is added to the number of sequences ending in q. Second, incoming arcs at q′sont redirected to q. If such arcs already exist, the number of passing sequences in each incoming arc is added to the existing arc. The Fold operation is a recursive function consisting in merging the successor nodes q′ into q and the corresponding successor nodes of q respectively.

In [24]:
def Original_Alergia(sequences, alpha):
    '''
    Original version of the Alergia Algorithm
    
    Arguments: 
    sequences -- the sequences to build the Trie and then convert it in a PFA
    alpha -- an int between 0 and 1. It represents a parameter of the Hoeffding bounds. 
    
    Return:
    A DPA according to the sequences we have
    '''
    print('Starting building corresponding Trie \n')
    trie = PrefixTree()
    trie.alphabet = list({l for word in sequences for l in word})
    for subseq in sequences:
        trie.insert(subseq)
    print(trie.size())
    #print(trie.leaf())
    #initial = trie.size()
    red = []
    red.append(trie.root)
    blue = []
    for x in trie.root.children[1:]:
        if(x[2] not in blue):
            blue.append(x[2])
    t0 = 0.5
    
    print('Running Alergia on trie \n')
    while(len(blue) > 0):
        qb = blue[0]
        blue.remove(qb)
        promote = True
        if(qb.get_frequency() >= t0):
            for qr in red:
                if(Original_AlergiaCompatible(trie, qr, qb, alpha)):
                    #print('Merge accepted...')
                    #print(qr)
                    #print(qb)
                    trie = Original_MergeFold(trie, qr, qb)
                    promote = False
                    for child in qr.children[1:]:    
                        if(child[2] not in blue and child[2].state != qr.state and child[2] not in red):
                            blue.append(child[2])
                    break
        
            if(promote == True):
                #print('No merge possible...')
                red.append(qb)
                for child in qb.children[1:]:
                    if(child[2] not in blue and child[2] not in red):
                        blue.append(child[2])
        else:
            continue
    
    trie.transform2proba()
    print('Algo done')
    return trie

def Original_AlergiaCompatible(trie, qr, qb, alpha):
    '''
    Sub-method of the Alergia main one. It tells us if 2 nodes (a red and a blue) are compatible
    
    Arguments:
    trie -- the current trie we are working on
    qr -- a red node 
    qb -- a blue node
    alpha -- parameter alpha mentionned above
    
    Return:
    a boolean saying whether or not the 2 nodes in arguments are compatible
    '''
    correct = True
    if(Original_AlergiaTest(qr.children[0][1], qr.get_frequency(), qb.children[0][1], qb.get_frequency(), alpha) == False):
        correct = False
    
    for a in trie.alphabet:
        for children_r in qr.children:
            for children_b in qb.children:
                if(a == children_r[1] and a == children_b[1]):
                    if(Original_AlergiaTest(children_r[2], children_r.get_frequency(), children_b[2], children_b.get_frequency(), alpha) == False):
                        correct = False
    return correct
  
def Original_AlergiaTest(f1, n1, f2, n2, alpha):
    '''
    Sub-method of the Alergia compatible method. It computes the hoeffding bounds and compare it to the gamma threshold
    
    Arguments:
    f1 -- incoming frequency of the node 1
    n1 -- number of times a word ends in node 1
    f2 -- incoming frequency of the node 2
    n2 -- number of times a word ends in node 2
    1 & 2 refers to the nodes we want to compare (i.e a node Red and a node Blue)
    alpha -- same parameter alpha as mentionned
    
    Return:
    True if gamma < hoeffding (then, the nodes are compatible)
    False if not
    '''
    gamma = math.fabs(f1/n1 - f2/n2)
    root = math.sqrt(1/(2*math.log(2/alpha)))
    summ = (1/math.sqrt(n1)) + (1/math.sqrt(n2))
    hoeffding = root * summ
    return gamma < hoeffding

def Original_MergeFold(trie, qr, qb): 
    '''
    MERGE OPERATION: 
    To merge a BLUE node into a RED node, all the ingoing links of BLUE node are redirected to the RED node. 
    If the RED node already has an ingoing link with the same item attribute as a redirected link, then only the frequencies are added. 
    Next, the BLUE node attribute is added to the RED node attribute.
    
    FOLD OPERATION:
    When a BLUE node merges into a RED node, all children of BLUE node are recursively merged into children of the RED node with the same item attribute link.
    If the link with an item attribute doesn't exist in the RED node, a new child is inserted.
    
    Arguments:
    trie -- the current Trie we are working on
    qr -- the red node 
    qb -- the blue node to merge
    '''
    q = qb.previous[0]
    
    if qb.text[-1] not in qr.text:
        qr.text = qr.text+ ', ' + qb.text[-1]
        
    for child in q.children:
        if(child[2].state == qb.state):
            child[2] = qr 
            
    qr.children[0][1] += qb.children[0][1]
    
    words = trie.starts_with('', current = qb)
    
    for word in words:
        trie.insert(word)
        
    return trie

In [25]:
%time dfa = Original_Alergia(seq, 0.5)
print(dfa.size())

Starting building corresponding Trie 

115
Running Alergia on trie 

Algo done
Wall time: 7 ms
11


In [26]:
def Relaxed_Alergia(sequences, alpha):
    '''
    Our version of the Alergia algorithm. It tries to recursively merge all the pairs of nodes, from the root to the leaves.
    Unlike the original version, the difference is that it does not test the compatibility of children of these pairs of nodes.
    
    Arguments: 
    sequences -- the sequences to build the Trie and then convert it in a PFA
    alpha -- an int between 0 and 1. It represents a parameter of the Hoeffding bounds. 
    
    Return:
    A DPA according to the sequences we have
    '''
    
    print('Starting building corresponding Trie \n')
    
    trie = PrefixTree()
    
    trie.alphabet = list({l for word in sequences for l in word})
    
    for subseq in sequences:
        trie.insert(subseq)
        
    #print(trie.size())
    #print(trie.leaf())
    #initial = trie.size()
    red = []
    
    red.append(trie.root)
    
    blue = []
    
    for x in trie.root.children[1:]:
        if(x[2] not in blue):
            blue.append(x[2])
            
    t0 = 0.5
    
    print('Running Alergia on trie \n')
    
    while(len(blue) > 0):
        qb = blue[0]
        blue.remove(qb)
        promote = True
        if(qb.get_frequency() >= t0):
            for qr in red:
                if(AlergiaCompatible(trie, qr, qb, alpha)):
                    #print('Merge accepted...')
                    #print(qr)
                    #print(qb)
                    trie = MergeFold(trie, qr, qb)
                    promote = False
                    for child in qr.children[1:]:    
                        if(child[2] not in blue and child[2].state != qr.state and child[2] not in red):
                            blue.append(child[2])
                    break
        
            if(promote == True):
                #print('No merge possible...')
                red.append(qb)
                for child in qb.children[1:]:
                    if(child[2] not in blue and child[2] not in red):
                        blue.append(child[2])
        else:
            continue
    
    trie.transform2proba()
    print('Algo done')
    return trie

def AlergiaCompatible(trie, qr, qb, alpha):
    '''
    Sub-method of the Alergia main one. It tells us if 2 nodes (a red and a blue) are compatible
    
    Arguments:
    trie -- the current trie we are working on
    qr -- a red node 
    qb -- a blue node
    alpha -- parameter alpha mentionned above
    
    Return:
    a boolean saying whether or not the 2 nodes in arguments are compatible
    '''
    correct = True
    
    if(AlergiaTest(qr.children[0][1], qr.get_frequency(), qb.children[0][1], qb.get_frequency(), alpha) == False):
        correct = False
    
    return correct
  
def AlergiaTest(f1, n1, f2, n2, alpha):
    '''
    Sub-method of the Alergia compatible method. It computes the hoeffding bounds and compare it to the gamma threshold
    
    Arguments:
    f1 -- incoming frequency of the node 1
    n1 -- number of times a word ends in node 1
    f2 -- incoming frequency of the node 2
    n2 -- number of times a word ends in node 2
    1 & 2 refers to the nodes we want to compare (i.e a node Red and a node Blue)
    alpha -- same parameter alpha as mentionned
    
    Return:
    True if gamma < hoeffding (then, the nodes are compatible)
    False if not
    '''
    gamma = math.fabs(f1/n1 - f2/n2)
    root = math.sqrt(1/(2*math.log(2/alpha)))
    summ = (1/math.sqrt(n1)) + (1/math.sqrt(n2))
    hoeffding = root * summ
    return gamma < hoeffding

def MergeFold(trie, qr, qb): 
    '''
    MERGE OPERATION: 
    To merge a BLUE node into a RED node, all the ingoing links of BLUE node are redirected to the RED node. 
    If the RED node already has an ingoing link with the same item attribute as a redirected link, then only the frequencies are added. 
    Next, the BLUE node attribute is added to the RED node attribute.
    
    FOLD OPERATION:
    When a BLUE node merges into a RED node, all children of BLUE node are recursively merged into children of the RED node with the same item attribute link.
    If the link with an item attribute doesn't exist in the RED node, a new child is inserted.
    
    Arguments:
    trie -- the current Trie we are working on
    qr -- the red node 
    qb -- the blue node to merge
    '''
    q = qb.previous[0]
    
    if qb.text[-1] not in qr.text:
        qr.text = qr.text+ ', ' + qb.text[-1]
        
    for child in q.children:
        if(child[2].state == qb.state):
            child[2] = qr 
            
    qr.children[0][1] += qb.children[0][1]
    
    words = trie.starts_with('', current = qb)
    
    for word in words:
        trie.insert(word)
        
    return trie

In [27]:
%time dfa_2 = Relaxed_Alergia(seq, 0.5)
print(dfa_2.size())

Starting building corresponding Trie 

Running Alergia on trie 

Algo done
Wall time: 5 ms
11


In [28]:
dfa_2.display()

symb:λ, 9, 1, 7, 8, 3, 0, 2, 4, 5 state:232 -> children:[['#', 0.11179173047473201, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['9', 0.0444104134762634, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['1', 0.06431852986217458, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['7', 0.03215926493108729, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['8', 0.18223583460949463, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['3', 0.06278713629402756, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['0', 0.1332312404287902, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['2', 0.08269525267993874, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['4', 0.12098009188361408, <__main__.TrieNode object at 0x000001D9EDCF20C8>], ['6', 0.06891271056661562, <__main__.TrieNode object at 0x000001D9EDC5DC08>], ['5', 0.09647779479326186, <__main__.TrieNode object at 0x000001D9EDCF20C8>]] 

symb:6 state:261 -> children:[['#', 0.26666666666666666, <__main__.TrieNode object at

In [29]:
dfa.display()

symb:λ, 9, 1, 7, 8, 3, 0, 2, 4, 5 state:116 -> children:[['#', 0.11179173047473201, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['9', 0.0444104134762634, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['1', 0.06431852986217458, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['7', 0.03215926493108729, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['8', 0.18223583460949463, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['3', 0.06278713629402756, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['0', 0.1332312404287902, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['2', 0.08269525267993874, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['4', 0.12098009188361408, <__main__.TrieNode object at 0x000001D9EDCE9A48>], ['6', 0.06891271056661562, <__main__.TrieNode object at 0x000001D9EDC57B08>], ['5', 0.09647779479326186, <__main__.TrieNode object at 0x000001D9EDCE9A48>]] 

symb:6 state:145 -> children:[['#', 0.26666666666666666, <__main__.TrieNode object at